### Idea General:
#### Usando datos de compras de hoteles (de Pentaho), adecuar el sorting de hoteles a las características más relevantes de cada tipo de búsqueda, buscando maximizar la probabilidad de compra. Usar variables como día de ci, ratio finde, destino, traveler type, gb/night

In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import tree, naive_bayes, grid_search

from sklearn.grid_search import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing

from sklearn.cluster import KMeans

from scipy import stats

import random
import re
import time
import datetime
from datetime import datetime, timedelta, date

from time import time
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits # trae un dataset para usar como ejemplo
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
# importo el csv y le pido que directamente me pase a formato de fecha determinadas columnas
BD= pd.read_csv("sortHotels.csv", delimiter=";", decimal=".", parse_dates=['id_dim_tiempo_checkin', 'id_dim_tiempo_checkout',
                                                                          'id_dim_tiempo_reserva']  )

In [ ]:
# paso a dataframe y veo que columnas traje de Pentaho
BD = DataFrame(BD)

BD.columns

In [ ]:
# truncate la duracion ya que la uso mas adelante como integer, por las dudas
BD['duracion'] = (BD['duracion']).apply(np.floor)

In [ ]:
# calculo la anticipacion de compra
BD['anticipacion'] = BD['id_dim_tiempo_checkin'] - BD['id_dim_tiempo_reserva']

# paso del formato timedelta (diferencia de dias) a integer
BD['anticipacion'] = BD['anticipacion'].dt.days
# ver https://docs.python.org/3/library/datetime.html#timedelta-objects

In [ ]:
# calculo gasto final por noche por habitacion
BD['gb_roomnight'] = BD['gb'] / BD['canthabitaciones'] / BD['duracion']

In [ ]:
# veo que hay en tipos de cliente
# 4: "AF" ; 5:"B2B" ; 6:"B2C" ; 0:"N/A" ; 7:"eargolo@hotmail.com"
 
#BD['id_dim_tipocliente'].describe()

# filtro la data. me quedo solo con Arg y con clientes no-agencia ni corporate
BD = BD[(BD['id_dim_pais']==22) & (BD['id_dim_tipocliente']==6)]

In [ ]:
# Traveler Type

# Create a list to store the data
traveler_type = []

# For each row in the bookings table,
for index, row in BD.iterrows(): # por que iterrows?? ver http://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe
    if row['Cantidad.Adt'] == 1 and row['Cantidad.Chd'] == 0 and row['Cantidad.Inf'] == 0:
        traveler_type.append("single")
    elif row['Cantidad.Adt'] == 2 and row['Cantidad.Chd'] == 0 and row['Cantidad.Inf'] == 0:
        traveler_type.append("couple")
    elif row['Cantidad.Adt'] > 2 and row['Cantidad.Chd'] == 0 and row['Cantidad.Inf'] == 0:
        traveler_type.append("adult group")
    else:
        traveler_type.append("family")

# Create a column from the list
BD['traveler'] = traveler_type

In [ ]:
### TAG MAILS CORPORATE Y CONSUMER

#Index de mail type
BD['mail_type'] = BD.semailprincipal.str.contains('yahoo|gmail|uol|outlook|hotmail|live|fibertel|arnet|bol.com|ig.com|superig|terra|globo|outlook|icloud|me.com|speedy|arnet|msn|ciudad|ymail|aol|superig|ibest', 
             flags=re.IGNORECASE, regex=True, na=False)

# mas sobre regular expressions: https://docs.python.org/3/library/re.html#module-re

print BD['mail_type'].describe() # el 84% de las observaciones es de alguna cuenta de gmail, hotmail, yahoo, etc.

# Pongo un tag
# Create a list to store the data
mail_classif = []

# For each row in the bookings table,
for index, row in BD.iterrows(): # por que iterrows?? ver http://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe
    if row['mail_type'] == True:
        mail_classif.append("consumer_mail")
    else:
        mail_classif.append("corporate_mail")

# Create a column from the list
BD['mail_type'] = mail_classif

### Limpio outliers

In [ ]:
BD.describe()

In [ ]:
# si la anticipacion es -1, pongo 0. Si es <1, lo excluyo
BD['anticipacion'] = BD['anticipacion'].apply(lambda x: 0 if (x == -1) else x)
BD = BD.loc[BD['anticipacion'] >= 0]

In [ ]:
# elijo que features / columnas quiero limpiar de outliers y cuales no hace falta y quiero conservar
columns = BD.columns
features_w_outliers = [column for column in columns if column in ('anticipacion','duracion','gb_roomnight',
                                                                  'Cantidad.Adt','Cantidad.Chd','Cantidad.Inf' )]
other_columns_to_keep = [column for column in columns if column in ('canthabitaciones', 'id_dim_tiempo_checkin',
                                                                  'id_dim_tiempo_checkout', 'id_dim_tiempo_reserva', 'gb',
                                                                  'mail_type', 'traveler', 'hoteldespegarid')]

In [ ]:
# funcion que me deja solo las columnas que me interesan y a la vez saca outliers de las que elijo

def filterOutliers(bookings, features_to_filter, other_feats_to_keep):
    
    #print bookings[features_to_filter].describe()
    
    for column in bookings:
        if column in features_to_filter:
            X_col = bookings[column] # se queda solo con las columnas que elegi para evaluar outliers
            # se queda solo con las obs de TODAS LAS COLUMNAS de bookings donde esas columnas cumplen la siguiente condicion.
            bookings = bookings[np.abs(X_col - X_col.mean()) <= (3 * X_col.std())] 
            
    return bookings[features_to_filter + other_feats_to_keep]

data = filterOutliers(BD, features_w_outliers, other_columns_to_keep)


# ### Variables de fechas

In [ ]:
def funcMagica(row):
    if row['dia_compra'] == 7:
        return 1
    elif row['dia_compra'] + row['resto'] == 7:
        return 1
    elif row['dia_compra'] + row['resto']  > 7:
        return 2
    else:
        return 0

def ratioFinde(actions):
    
    finde_columns = DataFrame()
    
    finde_columns['id_dim_tiempo_reserva'] =  actions['id_dim_tiempo_reserva']
    finde_columns['duracion'] = actions['duracion']
    
    # esto me dice el numero de dia de semana, del 0 al 6, empezando por el lunes = 1 y terminando por domingo = 7
    # date.isoweekday() returns the day of the week as an integer, where Monday is 1 and Sunday is 7.
    # como no me funciona isoweekday uso weekdayy sumo 1
    finde_columns['dia_compra'] = finde_columns['id_dim_tiempo_reserva'].dt.weekday + 1
    
    # cuantas semanas completas de 7 dias (con dos dias de finde) hay en la duracion del viaje?
    # trunc(duracion/7)
    finde_columns['semanas_completas'] = (finde_columns['duracion']/7).apply(np.floor)
    
    # resto de dias fuera de semanas completas:
    finde_columns['resto'] = finde_columns['duracion'] - finde_columns['semanas_completas'] * 7
    
    # esto es una adaptacion de esto http://stackoverflow.com/questions/18194404/create-column-with-elif-in-pandas
    finde_columns['dias_finde_extra'] = finde_columns.apply(funcMagica, axis=1)
   
    # ESTA SOLIA SER LA PAPA PERO NO ME SALIO Y tERMINE USANDO LA FUNCMAGICA
    #   for index, row in finde_columns.iterrows():
#       if row['dia_compra'] == 7:
#           row['dias_finde_extra'] = 1
#      elif row['dia_compra'] + row['resto'] == 7:
#           row['dias_finde_extra'] = 1
#      elif row['dia_compra'] + row['resto'] > 7:
#           row['dias_finde_extra'] = 2
#       else:
#           row['dias_finde_extra'] = 0

    # Saco el numero de dias de finde (dos por semana completa, mas los dias extra)
    finde_columns['dias_finde'] = finde_columns['semanas_completas'] * 2 + finde_columns['dias_finde_extra'] 

    # saco el ratio "dias de finde" / "duracion" del viaje
    finde_columns['ratio_finde'] = finde_columns['dias_finde'] / finde_columns['duracion']

    return actions.join(DataFrame(finde_columns['ratio_finde']))

    #return finde_columns
    

    data = ratioFinde(data)

In [ ]:
#Se aplica a cada columna de fecha de la que quiero saber el dia de la semana

def dayOfWeek(date_series): 

    nameday_series = []
    
    weekday_series = date_series.dt.weekday + 1

    for index, value in weekday_series.iteritems():
        if value == 1:
            nameday_series.append("Monday")
        elif value == 2:
            nameday_series.append("Tuesday")
        elif value == 3:
            nameday_series.append("Wednesday")
        elif value == 4:
            nameday_series.append("Thursday")
        elif value == 5:
            nameday_series.append("Friday")
        elif value == 5:
            nameday_series.append("Saturday")        
        else:
            nameday_series.append("Sunday")
    
    return nameday_series

In [ ]:
# aplico la funcion dayOfWeek para obtener el nombre del dia de la semana en las columnas de fecha
data['dia_sem_reserva'] = dayOfWeek(data['id_dim_tiempo_reserva'])
data['dia_sem_ci'] = dayOfWeek(data['id_dim_tiempo_checkin'])
data['dia_sem_co'] = dayOfWeek(data['id_dim_tiempo_checkout'])

#### Variables categóricas a numéricas

In [ ]:
# El problema es que k-means necesita computar means. no va para variables categoricas. 
# k-modes si pero cambia el resultado dependiendo del punto de partida.
# asi que paso a binomial. A mano. arbitrario

# CAMBIO EL DIA DE LA SEMANA POR FINDE/NO FINDE
#Se aplica a cada columna de fecha luego de aplicar la funcion del dia de la semana

def diaFinde(dias_de_finde, day_name_series): 

    findeday_series = day_name_series.apply(lambda x: 1 if (x in dias_de_finde) else 0)
            
    return findeday_series

# aplico la funcion dayOfWeek para obtener el nombre del dia de la semana en las columnas de fecha
# compra_finde = si compra sabado o domingo
# ci_finde = si hace check in jueves, viernes o sabado
# co_finde = si hace check out sabado, domingo o lunes

data['reserva_finde'] = diaFinde(['Saturday', 'Sunday'], data['dia_sem_reserva'])
data['ci_finde'] = diaFinde(['Thursday', 'Friday', 'Saturday'], data['dia_sem_ci'])
data['co_finde'] = diaFinde(['Saturday', 'Sunday', 'Monday'], data['dia_sem_co'])

In [ ]:
# CAMBIO MAIL TYPE POR CONSUMER/NO CONSUMER
data['consumer_mail'] = data['mail_type'].apply(lambda x: 1 if (x == "consumer_mail") else 0)

In [ ]:
# CAMBIO TRAVELER POR 3 DUMMIES: COUPLE/NOT COUPLE y SINGLE/NOT SINGLE y FAMILY/ NOT FAMILY
data['couple'] = data['traveler'].apply(lambda x: 1 if (x == "couple") else 0)
data['single'] = data['traveler'].apply(lambda x: 1 if (x == "single") else 0)
data['family'] = data['traveler'].apply(lambda x: 1 if (x == "family") else 0)

In [ ]:
data

#### Dataset final

In [ ]:
#elijo las variables de mi modelo
columns = data.columns
features_for_model = [column for column in columns if column in ('anticipacion','duracion','gb_roomnight','mail_type'
                                                                'traveler', 'ratio_finde', 'reserva_finde', 'ci_finde',
                                                                 'co_finde', 'consumer_mail', 'couple', 'single', 'family')]   
Y = data['hoteldespegarid'].values
X = data[features_for_model]

### K Time

In [ ]:
#### Primero creo las categorías con K-means, después clasifico los hotel ids en categorías.

__init__(n_clusters=10, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', 
         verbose=0, random_state=None, copy_x=True, n_jobs=1)

hpc = X.values

kmeans = KMeans()
kmeans.fit(hpc)

In [ ]:
# Comparo métodos de K Means

np.random.seed(42)

digits = X.as_matrix()

In [ ]:
X_scaled = preprocessing.scale(digits)

In [ ]:
X_scaled

In [ ]:
#http://blog.josephmisiti.com/help-commands-for-doing-machine-learning-in-python
    
n_samples, n_features = data.shape
n_digits = len(np.unique(X_scaled.target))
labels = X_scaled.target

sample_size = 300

print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))


print(79 * '_')
print('% 9s' % 'init'
      '    time  inertia    homo   compl  v-meas     ARI AMI  silhouette')

def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
              name="k-means++", data=data)

bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
              name="random", data=data)
# in this case the seeding of the centers is deterministic, hence we run the
# kmeans algorithm only once with n_init=1
pca = PCA(n_components=n_digits).fit(data)
bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
              name="PCA-based",
              data=data)
print(79 * '_')

In [ ]:
# establezco el criterio de clasificación

#classifier = tree.DecisionTreeClassifier()

# Set the parameters by cross-validation
tuned_parameters = {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'max_depth' : [2, 3, 30] }
                    
score = 'recall'

classifier = GridSearchCV(tree.DecisionTreeClassifier(), tuned_parameters, score, cv = 3)

In [ ]:
classifier = classifier.fit(X, Y)
y_predict = classifier.predict(X)

In [ ]:
BD['id_dim_tiempo_reserva'].__class__.__name__

In [ ]:
data.to_csv("dataSortHotels.csv", sep=',')

In [ ]:
# ver para cuantas ventas tengo tipo de viajero, duracion y dia de check in
count_grupo = BD[['tipo viajero','duracion','Dia sem CI']].count()
count_grupo

In [ ]:
#me quedo con un % random para test y train
test = BD.sample(frac=0.1, replace=False)
train = BD.sample(frac=0.9, replace=False)

In [ ]:
np.abs(test['tipo viajero'])

In [ ]:
def loadmatrix(users, features):
    #Complete missing values
    
    users = users.fillna(0)


    print users.describe()

    for col in users:
        if col in features:
            X_col = users[col]
            users = users[np.abs(X_col - X_col.mean()) <= (3 * X_col.std())]
        
    print users.describe()
    
    
    y = users['BOUGHT'].values
    X = users[features]

In [ ]:
to_filter = ['tipo viajero','duracion','hoteldespegarid','anticipacion',
                'semailprincipal','id_dim_pais','Mes','Anio','Dia sem CI']

for column in to_filter:
       users = test[np.abs(test[column] )]

In [ ]:
def loadmatrix(users, features):
    #Complete missing values
    # me quedo solo con estas columnas
    to_filter = ['tipo viajero','duracion','hoteldespegarid','anticipacion',
                 'semailprincipal','id_dim_pais','Mes','Anio','Dia sem CI']
    for column in to_filter:
        users = users[np.abs(users[column] )]
    for column in to_filter:
        users = users[users[columns]]
    y = users['Dia sem CI'].values
    X = users[features]
    return X, y

In [ ]:
columns = train.columns
features = [column for column in columns if column in ('Cantidad.Adt','Menores','tipo viajero','duracion','hoteldespegarid',
                                                       'anticipacion','id_dim_tiempo_checkin','id_dim_tiempo_checkout',
                                                       'semailprincipal','id_dim_tiempo_reserva','id_dim_pais','transaction_code',
                                                       'Mes','Anio','mes y anio CI','trimestre y anio')]

In [ ]:
features

In [ ]:
features = ['Cantidad.Adt','Menores','tipo viajero','duracion','hoteldespegarid','anticipacion','id_dim_tiempo_checkin',
            'id_dim_tiempo_checkout','semailprincipal','id_dim_tiempo_reserva','id_dim_pais','transaction_code',
            'Mes','Anio','mes y anio CI','trimestre y anio']

In [ ]:
features

In [ ]:
X_train, y_train = loadmatrix(train, features)
X_test, y_test = loadmatrix('Dia sem CI', features)

In [ ]:
X_train, y_train = loadmatrix(train, features)
X_test, y_test = loadmatrix(test, features)

#### Extras

In [ ]:
### SEPARO MAILS CORPORATE DE CONSUMER

# filtro emails para quedarme solo con los de gmail, uol, hotmail, yahoo para diferenciar lo corporate.
index_mails_regulares = BD.semailprincipal.str.contains('yahoo|gmail|uol|outlook|hotmail|live', 
             flags=re.IGNORECASE, regex=True, na=False)

# mas sobre regular expressions: https://docs.python.org/3/library/re.html#module-re

print index_mails.describe() # el 84% de las observaciones es de alguna cuenta de gmail, hotmail, yahoo, etc.

# me quedo solo con las compras de esos mails de consumers
BD_cons = BD[index_mails]

# me quedo solo con las compras de esos mails de consumers
BD_corp = BD[~index_mails]